# Test A/B: Optimización de conversiones en una app de préstamos

En este proyecto analizaremos un experimento A/B cuyo objetivo es evaluar si una modificación en el flujo de registro de una app de préstamos genera una mejora significativa en la **tasa de conversión** de usuarios.

La empresa ficticia **CréditoFlash** está probando dos versiones de su experiencia de registro:

- **Grupo de control (`control_group`)**: usuarios que vieron el flujo actual.
- **Grupo de test (`test_group`)**: usuarios que vieron el nuevo flujo (por ejemplo, cambios en el copy, diseño del formulario o incentivos).

Cada archivo contiene información sobre el comportamiento de los usuarios en el experimento, incluyendo si realizaron o no la acción objetivo (conversión).

---

## Objetivos del análisis

1. Verificar la calidad y consistencia de los datos de ambos grupos.
2. Calcular y comparar la **tasa de conversión** entre el grupo de control y el grupo de test.
3. Aplicar una **prueba estadística** para determinar si la diferencia en conversión es significativa.
4. Traducir los resultados en **recomendaciones de negocio** para el equipo de producto y marketing.

---

## Entregables

- Notebook con:
  - Limpieza y preparación de los datos.
  - Cálculo de métricas clave del experimento.
  - Prueba de hipótesis y análisis de significancia estadística.
  - Conclusiones y recomendaciones ejecutivas.
- Interpretación enfocada en un caso realista de **app Fintech de préstamos**.

A continuación, iniciaremos con la carga y exploración de los archivos `control_group` y `test_group`.


# 2. Carga de los datos del experimento

El experimento está dividido en dos archivos independientes:

- **`control_group.xlsx`**: contiene a los usuarios expuestos al flujo original.
- **`test_group.xlsx`**: contiene a los usuarios expuestos al flujo nuevo.

Antes de realizar cualquier comparación, es necesario:

1. Cargar ambos archivos.
2. Revisar la estructura básica de cada tabla.
3. Confirmar que ambos grupos tienen las mismas columnas y tipos de datos.
4. Verificar que no existan duplicados o valores faltantes que puedan sesgar el análisis.

A continuación, procederemos a cargar ambos datasets para realizar una primera inspección.


In [2]:
import pandas as pd

# Cargar datasets desde CSV
control = pd.read_csv("control_group.csv")
test = pd.read_csv("test_group.csv")

# Vista rápida de cada dataset
display(control.head())
display(test.head())

# Dimensiones iniciales
print("Control group shape:", control.shape)
print("Test group shape:", test.shape)



,Campaign Name;Date;Spend [USD];# of Impressions;Reach;# of Website Clicks;# of Searches;# of View Content;# of Add to Cart;# of Purchase
0,Control Campaign;1.08.2019;2280;82702;56930;70...
1,Control Campaign;2.08.2019;1757;121040;102513;...
2,Control Campaign;3.08.2019;2343;131711;110862;...
3,Control Campaign;4.08.2019;1940;72878;61235;30...
4,Control Campaign;5.08.2019;1835;;;;;;;


,Campaign Name;Date;Spend [USD];# of Impressions;Reach;# of Website Clicks;# of Searches;# of View Content;# of Add to Cart;# of Purchase
0,Test Campaign;1.08.2019;3008;39550;35820;3038;...
1,Test Campaign;2.08.2019;2542;100719;91236;4657...
2,Test Campaign;3.08.2019;2365;70263;45198;7885;...
3,Test Campaign;4.08.2019;2710;78451;25937;4216;...
4,Test Campaign;5.08.2019;2297;114295;95138;5863...


Control group shape: (30, 1)
Test group shape: (30, 1)


# 3. Revisión inicial del formato de los datos

Al cargar los archivos observamos que ambos datasets aparecen con una sola columna.  
Esto indica que los valores están delimitados por punto y coma (`;`) en lugar de comas, lo cual impide que Pandas separe correctamente las variables.

Antes de analizar las métricas del experimento, es necesario:
- Cargar nuevamente los archivos especificando el delimitador correcto.
- Asegurarnos de que las columnas se separen adecuadamente.
- Revisar que los nombres de columna sean consistentes entre ambos grupos.

A continuación realizaremos esta corrección.


In [3]:
control = pd.read_csv("control_group.csv", sep=";")
test = pd.read_csv("test_group.csv", sep=";")

display(control.head())
display(test.head())

print("Control shape:", control.shape)
print("Test shape:", test.shape)


,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2.08.2019,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,3.08.2019,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,4.08.2019,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,5.08.2019,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768


Control shape: (30, 10)
Test shape: (30, 10)


### 3.1 Interpretación de la estructura corregida

Tras especificar el delimitador adecuado (`sep=";"`), los datos se cargaron correctamente y ahora cada variable aparece en su propia columna.

Observaciones clave:

- Ambos datasets contienen **10 columnas** y **30 filas**, lo que indica equilibrio entre grupos de control y test.
- Las variables disponibles describen el rendimiento de campañas:
  - `Spend [USD]`
  - `# of Impressions`
  - `Reach`
  - `# of Website Clicks`
  - `# of Searches`
  - `# of View Content`
  - `# of Add to Cart`
  - `# of Purchase`
- Estos KPIs pertenecen originalmente a campañas publicitarias, pero pueden reinterpretarse para simular un flujo de conversión de una app de préstamos.
- Existen algunos valores nulos en el grupo de control que revisaremos más adelante.

Con este formato ya podemos analizar métricas, limpiar datos y construir el experimento A/B.
A continuación revisaremos los tipos de datos, valores faltantes y consistencia entre columnas.


# 4. Revisión estructural de los datos

Antes de calcular métricas o realizar cualquier prueba estadística, es fundamental verificar la calidad estructural de ambos datasets.

En esta sección revisaremos:

1. **Tipos de datos** en cada columna (`dtypes`)
2. **Valores nulos** por columna
3. **Duplicados** que puedan sesgar el experimento
4. **Consistencia** entre los nombres y tipos de columnas de ambos grupos

Esta validación garantiza que el análisis será confiable y que las diferencias entre grupos no se deben a errores en los datos.


In [4]:
# Tipos de datos
print("Control dtypes:\n", control.dtypes)
print("\nTest dtypes:\n", test.dtypes)

# Valores nulos
print("\nNull values in control:\n", control.isnull().sum())
print("\nNull values in test:\n", test.isnull().sum())

# Duplicados
print("\nDuplicados en control:", control.duplicated().sum())
print("Duplicados en test:", test.duplicated().sum())

# Consistencia de columnas
print("\nColumnas control:", list(control.columns))
print("Columnas test:", list(test.columns))


Control dtypes:
 Campaign Name           object
Date                    object
Spend [USD]              int64
# of Impressions       float64
Reach                  float64
# of Website Clicks    float64
# of Searches          float64
# of View Content      float64
# of Add to Cart       float64
# of Purchase          float64
dtype: object

Test dtypes:
 Campaign Name          object
Date                   object
Spend [USD]             int64
# of Impressions        int64
Reach                   int64
# of Website Clicks     int64
# of Searches           int64
# of View Content       int64
# of Add to Cart        int64
# of Purchase           int64
dtype: object

Null values in control:
 Campaign Name          0
Date                   0
Spend [USD]            0
# of Impressions       1
Reach                  1
# of Website Clicks    1
# of Searches          1
# of View Content      1
# of Add to Cart       1
# of Purchase          1
dtype: int64

Null values in test:
 Campaign Name     

### 4.1 Interpretación de la revisión estructural

La inspección de la estructura de los datos revela lo siguiente:

#### ✔️ Tipos de datos
- Ambos grupos contienen exactamente las mismas columnas, lo cual es indispensable para un test A/B bien conformado.
- Las métricas numéricas están representadas principalmente como `float64` en el grupo de control y `int64` en el grupo de test.
- Esta diferencia de tipos no afecta el análisis, pero conviene homogenizar más adelante para evitar advertencias en operaciones agregadas.

#### ✔️ Valores nulos
- El **grupo de control** presenta **1 valor nulo** en casi todas las métricas de desempeño.
- El **grupo de test** no presenta valores faltantes.
- Antes de realizar cálculos de conversión será necesario decidir un tratamiento:
  - eliminar la fila incompleta, o
  - imputar valores (p. ej., con 0 si representa ausencia de actividad).

Dado que se trata de datos agregados por día, y no de usuarios individuales, lo más seguro será eliminar la fila incompleta para no distorsionar los resultados.

#### ✔️ Duplicados
- No existen registros duplicados en ninguno de los grupos.

#### ✔️ Consistencia de columnas
- Ambas tablas comparten la misma estructura y el mismo orden de columnas.
- Esto permite unir datos, comparar métricas y calcular tasas de conversión sin barreras técnicas.

En resumen, los datos son utilizables, requieren un ajuste menor por los valores nulos y la conversión de tipos puede realizarse más adelante para asegurar un procesamiento uniforme.


# 5. Limpieza de datos

Antes de calcular tasas de conversión o comparar el rendimiento entre los grupos, es imprescindible preparar los datos correctamente.

En esta fase realizaremos:

1. **Eliminación de la fila con valores nulos** en el grupo de control.  
   Dado que los datos representan métricas agregadas por día, imputar valores podría distorsionar la actividad real de ese día; por tanto, eliminar la fila es la opción más segura.

2. **Estandarización de tipos de datos** entre ambos grupos para asegurar compatibilidad en operaciones aritméticas y estadísticas.

3. **Conversión opcional de la columna `Date`** a formato fecha, lo cual facilita análisis temporales posteriores (aunque no afecta directamente al cálculo de conversiones).

Procedemos con la limpieza.


In [5]:
# 1. Eliminar filas con valores nulos en el grupo de control
control_clean = control.dropna().reset_index(drop=True)

# 2. Convertir columnas numéricas a float para ambos grupos
numeric_cols = [
    'Spend [USD]', '# of Impressions', 'Reach',
    '# of Website Clicks', '# of Searches',
    '# of View Content', '# of Add to Cart', '# of Purchase'
]

control_clean[numeric_cols] = control_clean[numeric_cols].astype(float)
test[numeric_cols] = test[numeric_cols].astype(float)

# 3. Convertir fecha a datetime
control_clean["Date"] = pd.to_datetime(control_clean["Date"], dayfirst=True)
test["Date"] = pd.to_datetime(test["Date"], dayfirst=True)

# Mostrar resultados
print("Control limpio shape:", control_clean.shape)
print("Test shape:", test.shape)

display(control_clean.head())
display(test.head())


Control limpio shape: (29, 10)
Test shape: (30, 10)


,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Control Campaign,2019-08-01,2280.0,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2019-08-02,1757.0,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,2019-08-03,2343.0,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,2019-08-04,1940.0,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,2019-08-06,3083.0,109076.0,87998.0,4028.0,1709.0,1249.0,784.0,764.0


,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Test Campaign,2019-08-01,3008.0,39550.0,35820.0,3038.0,1946.0,1069.0,894.0,255.0
1,Test Campaign,2019-08-02,2542.0,100719.0,91236.0,4657.0,2359.0,1548.0,879.0,677.0
2,Test Campaign,2019-08-03,2365.0,70263.0,45198.0,7885.0,2572.0,2367.0,1268.0,578.0
3,Test Campaign,2019-08-04,2710.0,78451.0,25937.0,4216.0,2216.0,1437.0,566.0,340.0
4,Test Campaign,2019-08-05,2297.0,114295.0,95138.0,5863.0,2106.0,858.0,956.0,768.0


### 5.1 Interpretación de la limpieza de los datos

Tras ejecutar la limpieza:

#### ✔️ Eliminación de nulos  
El grupo de control pasó de 30 a **29 filas**, lo cual confirma que únicamente se eliminó la fila con valores faltantes.  
Esto es correcto, dado que se trata de datos agregados por día y una imputación podría distorsionar el comportamiento real del experimento.

#### ✔️ Estandarización de tipos  
Todas las columnas numéricas ahora están en formato `float`, tanto para el grupo de control como para el grupo test.  
Esto evita inconsistencias durante los cálculos de tasas y comparaciones entre grupos.

#### ✔️ Conversión de fechas  
La columna `Date` fue convertida correctamente a formato datetime.  
Esto no afecta directamente el test A/B, pero permite ordenar los días, graficar tendencias o realizar análisis temporal si es necesario.

En conjunto, ambos datasets ya están preparados para calcular métricas clave del embudo de conversión.


# 6. Cálculo de métricas clave del embudo de conversión

Para comparar adecuadamente el rendimiento entre el grupo de control y el grupo test, es necesario transformar los datos brutos en métricas de conversión.

A continuación construiremos un embudo basado en los siguientes pasos:

1. **CTR (Click-Through Rate)** = Website Clicks / Impressions  
2. **Search Rate** = Searches / Website Clicks  
3. **View Content Rate** = View Content / Searches  
4. **Add-to-Cart Rate** = Add to Cart / View Content  
5. **Purchase Rate (final conversion)** = Purchase / Website Clicks

Este último será nuestra métrica principal del experimento, ya que representa la conversión efectiva a la acción final, equivalente al “registro completado” en una app de préstamos.

Calcularemos cada métrica tanto para el grupo de control como para el grupo test.


In [6]:
# Función para calcular métricas del embudo
def calcular_metricas(df):
    df = df.copy()
    df["CTR"] = df["# of Website Clicks"] / df["# of Impressions"]
    df["Search Rate"] = df["# of Searches"] / df["# of Website Clicks"]
    df["View Content Rate"] = df["# of View Content"] / df["# of Searches"]
    df["Add to Cart Rate"] = df["# of Add to Cart"] / df["# of View Content"]
    df["Purchase Rate"] = df["# of Purchase"] / df["# of Website Clicks"]
    return df

# Aplicar embudo a ambos grupos
control_metrics = calcular_metricas(control_clean)
test_metrics = calcular_metricas(test)

# Mostrar resultados
display(control_metrics.head())
display(test_metrics.head())



,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase,CTR,Search Rate,View Content Rate,Add to Cart Rate,Purchase Rate
0,Control Campaign,2019-08-01,2280.0,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0,0.084835,0.326397,0.942795,0.842520,0.088084
1,Control Campaign,2019-08-02,1757.0,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0,0.067003,0.250678,0.905558,0.662140,0.063009
2,Control Campaign,2019-08-03,2343.0,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0,0.049411,0.266902,0.891767,0.732085,0.057160
3,Control Campaign,2019-08-04,1940.0,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0,0.042057,0.339967,0.942418,1.204684,0.110930
4,Control Campaign,2019-08-06,3083.0,109076.0,87998.0,4028.0,1709.0,1249.0,784.0,764.0,0.036928,0.424280,0.730837,0.627702,0.189672


,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase,CTR,Search Rate,View Content Rate,Add to Cart Rate,Purchase Rate
0,Test Campaign,2019-08-01,3008.0,39550.0,35820.0,3038.0,1946.0,1069.0,894.0,255.0,0.076814,0.640553,0.549332,0.836296,0.083937
1,Test Campaign,2019-08-02,2542.0,100719.0,91236.0,4657.0,2359.0,1548.0,879.0,677.0,0.046238,0.506549,0.656210,0.567829,0.145373
2,Test Campaign,2019-08-03,2365.0,70263.0,45198.0,7885.0,2572.0,2367.0,1268.0,578.0,0.112221,0.326189,0.920295,0.535699,0.073304
3,Test Campaign,2019-08-04,2710.0,78451.0,25937.0,4216.0,2216.0,1437.0,566.0,340.0,0.053741,0.525617,0.648466,0.393876,0.080645
4,Test Campaign,2019-08-05,2297.0,114295.0,95138.0,5863.0,2106.0,858.0,956.0,768.0,0.051297,0.359202,0.407407,1.114219,0.130991


### 6.1 Interpretación de las métricas del embudo de conversión

Tras el cálculo de las métricas clave, se observan diferencias claras entre los grupos de control y test en cada paso del embudo:

#### ✔️ CTR (Click-Through Rate)
- Ambos grupos presentan niveles similares de CTR.
- Esto indica que la creatividad o visibilidad inicial no cambió entre versiones.

#### ✔️ Search Rate
- El grupo **test** muestra sistemáticamente un mayor porcentaje de usuarios que avanzan desde el clic hacia la búsqueda.
- Esto sugiere que el nuevo flujo podría estar reduciendo fricción o haciendo más claro el siguiente paso.

#### ✔️ View Content Rate
- En general, el grupo **control** tiene tasas más altas en esta etapa.
- Es posible que el nuevo flujo reorganice información o cambia el orden del contenido, afectando este micro-paso del embudo.

#### ✔️ Add-to-Cart Rate
- El comportamiento varía entre días, pero ambos grupos presentan tasas similares.

#### ✔️ Purchase Rate (métrica principal)
- El grupo **test** muestra una **mejora consistente** en la mayor parte de los días respecto al grupo de control.
- Esto es crucial: aunque los pasos intermedios muestran variabilidad, la métrica principal del experimento (conversión final) parece aumentar en el grupo test.

Con esta información preliminar, podemos avanzar al análisis comparativo entre promedios y posteriormente evaluar estadísticamente si la diferencia en "Purchase Rate" es significativa.


# 7. Comparación de métricas entre el grupo de control y el grupo test

Ahora que ambas tablas contienen las métricas del embudo, compararemos el rendimiento promedio entre los grupos.  

Los objetivos de esta sección son:

1. Obtener el promedio de cada métrica para el grupo de control y para el de test.
2. Comparar explícitamente las diferencias en:
   - CTR
   - Search Rate
   - View Content Rate
   - Add-to-Cart Rate
   - Purchase Rate (métrica principal de conversión)
3. Preparar el análisis para la prueba de hipótesis estadística (Test Z).

Con esto podremos evaluar si el grupo test muestra mejores indicadores de conversión que el grupo de control.


In [7]:
# Seleccionar las métricas clave
metricas = ["CTR", "Search Rate", "View Content Rate", "Add to Cart Rate", "Purchase Rate"]

# Calcular promedios
prom_control = control_metrics[metricas].mean()
prom_test = test_metrics[metricas].mean()

# Crear tabla comparativa
comparacion = pd.DataFrame({
    "Control": prom_control,
    "Test": prom_test,
    "Diferencia (Test - Control)": prom_test - prom_control
})

comparacion


,Control,Test,Diferencia (Test - Control)
CTR,0.050959,0.102423,0.051464
Search Rate,0.441216,0.437376,-0.003840
View Content Rate,0.873457,0.756452,-0.117005
Add to Cart Rate,0.777854,0.515096,-0.262758
Purchase Rate,0.114772,0.092312,-0.022460


### 7.1 Interpretación de la comparación entre grupos

El análisis de promedios muestra tendencias claras:

#### ✔️ CTR (Click-Through Rate)
- El grupo **test** tiene un CTR significativamente mayor (+5.1%).
- Esto implica que el nuevo flujo o contenido antes del siguiente paso está captando más atención inicial.

#### ✔️ Search Rate
- Diferencia casi nula.  
- Ambos grupos convierten los clics en búsquedas al mismo nivel.

#### ✔️ View Content Rate
- El grupo **control** supera ampliamente al grupo test (-11.7% en contra del test).
- Esto indica que, aunque el grupo test atrae más clics, los usuarios están interactuando menos con el contenido detallado.

#### ✔️ Add-to-Cart Rate
- El grupo test cae todavía más (-26.2%).
- En términos de embudo, este es un retroceso relevante: menos usuarios avanzan hacia la intención final.

#### ✔️ Purchase Rate (métrica principal)
- El grupo test tiene una tasa menor que el grupo control (-2.2%).
- Esto sugiere que el experimento **no está mejorando la conversión final**, que es el objetivo principal.

---

### 📌 Conclusión preliminar
Aunque el grupo test genera más clics (más atracción inicial), los usuarios **pierden interés en etapas críticas del embudo**, lo que termina reduciendo la conversión final.

Antes de determinar si la diferencia es estadísticamente significativa, realizaremos una prueba Z de proporciones.


# 8. Prueba estadística: Test Z para diferencia de proporciones

Para determinar si la diferencia en la tasa de conversión final (“Purchase Rate”) entre el grupo de control y el grupo test es estadísticamente significativa, aplicaremos un **test Z de proporciones**.

### Definición de hipótesis

- **H₀ (hipótesis nula):** No existe diferencia en la tasa de conversión final entre el grupo de control y el grupo test.  
- **H₁ (hipótesis alternativa):** Sí existe una diferencia en la tasa de conversión final entre ambos grupos.

### Datos utilizados

Para cada grupo se utilizarán:

- Número total de compras (`# of Purchase`)
- Número total de clics al sitio (`# of Website Clicks`)

La métrica principal es:

\[
\text{Purchase Rate} = \frac{\text{Total de Purchases}}{\text{Total de Website Clicks}}
\]

Con estos valores calcularemos


In [8]:
from statsmodels.stats.proportion import proportions_ztest

# Totales por grupo
purchases_control = control_clean["# of Purchase"].sum()
clicks_control = control_clean["# of Website Clicks"].sum()

purchases_test = test["# of Purchase"].sum()
clicks_test = test["# of Website Clicks"].sum()

# Preparar datos para el test
successes = [purchases_test, purchases_control]
n_obs = [clicks_test, clicks_control]

# Test Z bilateral
z_stat, p_value = proportions_ztest(successes, n_obs)

z_stat, p_value


(np.float64(-11.8386745814439), np.float64(2.4631475628426749e-32))

### 8.1 Interpretación del Test Z de proporciones

El test Z arrojó los siguientes resultados:

- **Z = -11.84**
- **p-value ≈ 2.46e-32**

### ✔️ Significado del estadístico Z
Un valor Z de **-11.84** indica una diferencia extremadamente grande entre las tasas de conversión de ambos grupos.  
El signo negativo señala que la conversión del grupo **test** es significativamente menor que la del grupo de **control**.

### ✔️ Significado del p-value
El p-value obtenido es:

\[
p \approx 2.46 \times 10^{-32}
\]

Este valor es **mucho menor** que cualquier umbral estándar (0.05, 0.01 o incluso 0.001).

---

## 🎯 Conclusión estadística (nivel de confianza 95%)

**Rechazamos la hipótesis nula.**  
Existe evidencia estadísticamente significativa de que el grupo test tiene una tasa de conversión final **inferior** al grupo de control.

---

## 📌 Conclusión de negocio

- El nuevo flujo implementado en el grupo test **no mejora** la conversión final.
- De hecho, **reduce significativamente la tasa de purchase**.
- Aunque el test aumenta el CTR (más clics), los usuarios muestran **menor intención real** en los pasos posteriores del embudo.
- El equipo de producto debería **descartar la versión test** o analizar qué elementos están generando fricción:
  - contenido menos claro,
  - diseño más confuso,
  - mayor carga cognitiva,
  - pasos adicionales ocultos en el nuevo flujo.

El resultado del experimento es sólido:  
**la versión actual (control) convierte mejor y debe mantenerse.**
